In [1]:
import json
import GPy
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
from os.path import join, splitext
from os import listdir

In [2]:
data_path = '/home/rodrigo/Documents/curvas-de-luz-EL4106/OGLE_trainingDB/'
sub_class = 'CEPH/F'

In [3]:
lc_list = [splitext(file)[0] for file in sorted(listdir(join(data_path, sub_class))) if splitext(file)[1] == '.dat']

In [4]:
print(len(lc_list))
file = open('database.csv','a')
def smooth(x, window_len=11, window='hanning'):
    if window_len < 3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
    s=np.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')
    y=np.convolve(w/w.sum(),s,mode='valid')
    return y    

1482


In [5]:
for lc_index in range(len(lc_list)):
    
    data = np.loadtxt(join(data_path, sub_class, lc_list[lc_index]+'.time'))
    period = np.loadtxt(join(data_path, sub_class, lc_list[lc_index]+'.dat'))
    mjd, mag_r, err_r, mag_b, err_b = data[:, 0], data[:, 1], data[:, 2], data[:, 3], data[:, 4]
    """
    mjd_b = mjd
    mjd_r = mjd
    """
    mjd_b = mjd[err_b < 9.999]
    mjd_r = mjd[err_r < 9.999]
    mag_b = mag_b[err_b < 9.999]
    mag_r = mag_r[err_r < 9.999]
    err_b = err_b[err_b < 9.999]
    err_r = err_r[err_r < 9.999]

    mask = err_b < np.mean(err_b) + 3*np.std(err_b)
    mjd_b = mjd_b[mask]
    mag_b = mag_b[mask]
    err_b = err_b[mask]
    mask = err_r < np.mean(err_r) + 3*np.std(err_r)
    mjd_r = mjd_r[mask]
    mag_r = mag_r[mask]
    err_r = err_r[mask]
    
    phi = np.mod(mjd_b, period)/period
    I = np.argsort(phi)
    phi_rs = np.linspace(0, 1, num=200)
    mag_b_rs = np.interp(phi_rs, phi[I], mag_b[I])
    

    kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
    m = GPy.models.GPHeteroscedasticRegression(phi[:, np.newaxis], mag_b[:, np.newaxis], kernel)
    m['.*het_Gauss.variance'] = err_b[:, np.newaxis]**2 
    m.het_Gauss.variance.fix() 
    #m.het_Gauss.period.fix() 
    m.optimize()

    m.optimize(messages=True)
    m.optimize_restarts(num_restarts = 5)
    mu, var = m._raw_predict(phi_rs[:, np.newaxis])
    # File writer
    file.write(json.dumps(phi_rs.tolist())+";"+json.dumps(mu[:,0].tolist())+";"+sub_class+'\n')
    print(lc_index)
file.close()
    

    


Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s10  0001  -1.251838e+03   1.380761e-10 
    00s35  0004  -1.251838e+03   7.244849e-13 
Runtime:     00s35
Optimization status: Converged

Optimization restart 1/5, f = -1251.8381697737832
Optimization restart 2/5, f = -1251.8381697627046
Optimization restart 3/5, f = -1251.8381697565615
Optimization restart 4/5, f = -1251.8381697578334
Optimization restart 5/5, f = -1251.8381697501336
0
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s31  0004  -1.080550e+03   1.095072e-04 
Runtime:     00s31
Optimization status: Converged

Optimization restart 1/5, f = -1080.5497750167508
Optimization restart 2/5, f = -1080.5497752225613
Optimization restart 3/5, f = -1080.5497752164722
Optimization restart 4/5, f = -1080.5497752144872
Optimization restart 5/5, f = -1080.5497752017802
1
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f   

    00s15  0002  -1.115941e+03   1.037572e+05 
    01s43  0022  -1.284737e+03   7.013082e-07 
Runtime:     01s43
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSRCH'

Optimization restart 1/5, f = -1284.7366753024637
Optimization restart 2/5, f = -1284.7366753002286
Optimization restart 3/5, f = -1284.7366752469288
Optimization restart 4/5, f = -1284.7366753060617
Optimization restart 5/5, f = -1284.7366753056929
17
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s32  0005  -1.221714e+03   1.063492e-08 
Runtime:     00s32
Optimization status: Converged

Optimization restart 1/5, f = -1221.7142550182107
Optimization restart 2/5, f = -1221.7142576191357
Optimization restart 3/5, f = -1221.7142576074016
Optimization restart 4/5, f = -1221.714257598529
Optimization restart 5/5, f = -1221.714257605672
18
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s16  0002  -7.312734e+02   7.60469

    00s00  0000  -3.897897e+02   3.399334e-10 
    00s43  0007  -3.897897e+02   1.253039e-09 
Runtime:     00s43
Optimization status: Converged

Optimization restart 1/5, f = -389.78967524115706
Optimization restart 2/5, f = -389.7896749615411
Optimization restart 3/5, f = -389.789675178401
Optimization restart 4/5, f = -389.7896751750204
Optimization restart 5/5, f = -389.7896751479366
34
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s12  0002  -1.011420e+03   6.245308e+03 
    00s24  0004  -1.052634e+03   1.554527e-07 
Runtime:     00s24
Optimization status: Converged

Optimization restart 1/5, f = -1052.6342118512982
Optimization restart 2/5, f = -1052.6342118369255
Optimization restart 3/5, f = -1052.6342118435148
Optimization restart 4/5, f = -1052.634211200617
Optimization restart 5/5, f = -1052.6342118192697
35
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s33  0005  -8.434594

Optimization restart 1/5, f = -1393.7430767714823
Optimization restart 2/5, f = -1393.7430767470019
Optimization restart 3/5, f = -1393.743076763672
Optimization restart 4/5, f = -1393.7430767553956
Optimization restart 5/5, f = -1393.74307676222
51
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s26  0004  -1.291458e+03   2.033485e-08 
Runtime:     00s26
Optimization status: Converged

Optimization restart 1/5, f = -1291.4576975905018
Optimization restart 2/5, f = -1291.4576891082695
Optimization restart 3/5, f = -1291.457697575876
Optimization restart 4/5, f = -1291.4576975806733
Optimization restart 5/5, f = -1291.4576975752464
52
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s11  0003  -7.104258e+02   2.297210e-08 
    00s19  0005  -7.104258e+02   4.309681e-08 
    00s73  0022  -7.104258e+02   4.309252e-08 
Runtime:     00s73
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSR

    00s13  0002  -9.485322e+02   1.291431e+04 
    01s26  0022  -1.024913e+03   2.795589e-07 
Runtime:     01s26
Optimization status: Converged

Optimization restart 1/5, f = -1024.9133404187792
Optimization restart 2/5, f = -1024.9133403987225
Optimization restart 3/5, f = -1024.9133403712412
Optimization restart 4/5, f = -1024.913340391535
Optimization restart 5/5, f = -1024.913339739429
69
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s14  0002  -9.682075e+02   1.035734e-13 
Runtime:     00s14
Optimization status: Converged

Optimization restart 1/5, f = -968.2075340248189
Optimization restart 2/5, f = -968.2075340975794
Optimization restart 3/5, f = -968.20753402627
Optimization restart 4/5, f = -968.2075340232341
Optimization restart 5/5, f = -968.2075341154527
70
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s15  0002  -6.244940e+02   9.595518e+05 
    00s50  0007  -1.099506e+0

...

    00s44  0007  -1.405046e+03   1.147577e-08 
Runtime:     00s44
Optimization status: Converged

Optimization restart 1/5, f = -1405.0459959081202
Optimization restart 2/5, f = -1405.045995852405
Optimization restart 3/5, f = -1405.045995807404
Optimization restart 4/5, f = -1405.0459871660594
Optimization restart 5/5, f = -1405.0459958524887
1409
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s24  0005  -7.155969e+02   1.084602e-13 
Runtime:     00s24
Optimization status: Converged

Optimization restart 1/5, f = -715.5969204439975
Optimization restart 2/5, f = -715.5969203238596
Optimization restart 3/5, f = -715.5969204251469
Optimization restart 4/5, f = -715.5969204648104
Optimization restart 5/5, f = -715.5969204171197
1410
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s88  0015  -1.362734e+03   2.257940e-08 
Runtime:     00s88
Optimization status: Converged

Optimization restar

Optimization restart 4/5, f = -1236.136916524933
Optimization restart 5/5, f = -1236.136916588241
1426
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s18  0003  -1.199253e+03   1.523746e-09 
    01s21  0022  -1.199253e+03   3.754803e-08 
Runtime:     01s21
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSRCH'

Optimization restart 1/5, f = -1199.2531134607136
Optimization restart 2/5, f = -1199.253113417173
Optimization restart 3/5, f = -1199.253113434376
Optimization restart 4/5, f = -1199.2531134465455
Optimization restart 5/5, f = -1199.2531133794857
1427
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s19  0003  -1.342766e+03   2.895896e-09 
    00s73  0012  -1.342766e+03   8.824118e-10 
Runtime:     00s73
Optimization status: Converged

Optimization restart 1/5, f = -1342.7659182832667
Optimization restart 2/5, f = -1342.765917961585
Optimization restart 3/5, f = -1342.765918

Optimization restart 2/5, f = -626.6311882215596
Optimization restart 3/5, f = -626.6311880697406
Optimization restart 4/5, f = -626.6311880672249
Optimization restart 5/5, f = -626.6311881534778
1443
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s17  0003  -1.146677e+03   5.685839e-09 
    01s18  0022  -1.146677e+03   3.499442e-08 
Runtime:     01s18
Optimization status: Converged

Optimization restart 1/5, f = -1146.6774941114484
Optimization restart 2/5, f = -1146.6774941463132
Optimization restart 3/5, f = -1146.6774941177089
Optimization restart 4/5, f = -1146.6774941419465
Optimization restart 5/5, f = -1146.6774941679178
1444
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s13  0002  -2.288884e+02   9.125090e+04 
    01s06  0018  -5.241469e+02   6.759437e-05 
Runtime:     01s06
Optimization status: Converged

Optimization restart 1/5, f = -524.1468521762558
Optimization restart 

    00s19  0003  -9.354352e+02   5.012540e-10 
    01s01  0018  -9.354352e+02   1.725206e-10 
Runtime:     01s01
Optimization status: Converged

Optimization restart 1/5, f = -935.4352326098117
Optimization restart 2/5, f = -935.4352325958273
Optimization restart 3/5, f = -935.4352325182335
Optimization restart 4/5, f = -935.43523255726
Optimization restart 5/5, f = -935.4352325804869
1460
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s13  0002  -1.170743e+03   2.131624e-13 
Runtime:     00s13
Optimization status: Converged

Optimization restart 1/5, f = -1170.7425354096245
Optimization restart 2/5, f = -1170.7425353591798
Optimization restart 3/5, f = -1170.742535360906
Optimization restart 4/5, f = -1170.7425354008192
Optimization restart 5/5, f = -1170.7425350414505
1461
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s11  0002  -1.231657e+03   5.657335e-11 
Runtime:     00s11
Optim

    01s46  0022  -6.534583e+02   3.601379e-07 
Runtime:     01s46
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSRCH'

Optimization restart 1/5, f = -653.4583316649503
Optimization restart 2/5, f = -653.4583306772881
Optimization restart 3/5, f = -653.458328174381
Optimization restart 4/5, f = -653.4583311351441
Optimization restart 5/5, f = -653.4583317169611
1477
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s12  0004   1.173461e+03   1.388394e-07 
    00s18  0006   1.173461e+03   4.427952e-07 
    00s52  0022   1.173461e+03   4.427952e-07 
Runtime:     00s52
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSRCH'

Optimization restart 1/5, f = 1173.4611926494294
Optimization restart 2/5, f = 867.3487332445529
Optimization restart 3/5, f = 1192.8618408591205
Optimization restart 4/5, f = 1192.8618407728964
Optimization restart 5/5, f = 1192.861840549032
1478
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i    

In [6]:
print("Finished")

Finished
